# Demo Notebook for MLCommons Integration

#### [Download notebook](https://github.com/opensearch-project/opensearch-py-ml/blob/main/docs/source/examples/demo_ml_commons_integration.ipynb)


This notebook provides a walkthrough guidance for users to invoke MLCommons apis to upload ml models to opensearch cluster

Step 0: Import packages and set up client

Step 1: Upload NLP model from local file to Opensearch cluster

Step 2: Load Model

Step 3: Get Task

Step 4: Get Model

Step 5: Generate Sentence Embedding

Step 6: Unload Model

Step 7: Delete Model



## Step 0: Import packages and set up client
Install required packages for opensearch_py_ml.sentence_transformer_model
Install `opensearchpy` and `opensearch-py-ml` through pypi


In [1]:
!pip install opensearch-py opensearch-py-ml

In [2]:
!pip install deprecated

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [3]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings("ignore", message="Unverified HTTPS request")
from opensearchpy import OpenSearch

In [4]:
CLUSTER_URL = 'https://localhost:9200'

In [5]:
def get_os_client(cluster_url = CLUSTER_URL,
                  username='admin',
                  password='< admin password >'):
    '''
    Get OpenSearch client
    :param cluster_url: cluster URL like https://ml-te-netwo-1s12ba42br23v-ff1736fa7db98ff2.elb.us-west-2.amazonaws.com:443
    :return: OpenSearch client
    '''
    client = OpenSearch(
        hosts=[cluster_url],
        http_auth=(username, password),
        verify_certs=False
    )
    return client 

In [6]:
client = get_os_client()

/home/linuxbrew/.linuxbrew/opt/python@3.8/lib/python3.8/site-packages/opensearchpy/connection/http_urllib3.py:199: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


## Step 1: Upload NLP model from local file to Opensearch cluster

We can upload machine learning models to Opensearch cluster using MLCommons register_model api. In this demo we will show how can we upload model


###### From Opensearch 2.8, to register a model we need to have a model group. First we need to register a model group and use the model group id to register a model. For registering a model group we can look at this doc: 

https://github.com/opensearch-project/ml-commons/blob/2.x/docs/model_access_control.md#registering-a-model-group

In our following example, we created a group and using the group id to register a model.

`model_id = ml_client.register_pretrained_model(model_name = "huggingface/sentence-transformers/all-MiniLM-L12-v2", model_version = "1.0.1", model_format = "TORCH_SCRIPT", model_group_id = "d4hfsYgBFp6IJxCcqpwi", deploy_model=False, wait_until_deployed=False)`

###### From Opensearch 2.6, we introduced pre-trained models: https://opensearch.org/docs/latest/ml-commons-plugin/pretrained-models/


* One thing to remember, if we don't have any ml node then registering model might throw exception. In that case we need to update this setting: https://github.com/opensearch-project/ml-commons/blob/main/build.gradle#L46


In [7]:
from opensearch_py_ml.ml_commons import MLCommonClient
ml_client = MLCommonClient(client)

In [8]:
model_id = ml_client.register_pretrained_model(model_name = "huggingface/sentence-transformers/all-MiniLM-L12-v2", model_version = "1.0.1", model_format = "TORCH_SCRIPT", deploy_model=False, wait_until_deployed=False)

Model was registered successfully. Model Id:  t0jp24kBHUgZTTdfDMmm
t0jp24kBHUgZTTdfDMmm


We can also upload model from our own file system or URL. But to do that we need to update couple cluster settings:

To register from url: plugins.ml_commons.allow_registering_model_via_url
To register from file system: plugins.ml_commons.allow_registering_model_via_local_file

    By default, both of these values are `False`, we need to update to `True`


To demonstrate, we download the model zip file from the url: https://github.com/opensearch-project/ml-commons/raw/2.x/ml-algorithms/src/test/resources/org/opensearch/ml/engine/algorithms/text_embedding/all-MiniLM-L6-v2_torchscript_sentence-transformer.zip?raw=true

To upload model to the cluster, we need a zip file containing a torchScript file (.pt extension) and a tokenizer.json file. Please refer to the previous download. We also need a json file with defining the config information with following these request fields: 

https://opensearch.org/docs/latest/ml-commons-plugin/api/#request-fields

In [6]:

model_path = '/Volumes/workplace/upload_content/sentence-transformers_all-MiniLM-L6-v2-1.0.0-torch_script.zip'
model_config_path = '/Volumes/workplace/upload_content/all-MiniLM-L6-v2_torchscript.json'

"""
all-MiniLM-L6-v2_torchscript.json content:

{
    "name": "all-MiniLM-L6-v2",
    "version": 1,
    "model_format": "TORCH_SCRIPT",
    "model_config": {
        "model_type": "bert",
        "embedding_dimension": 384,
        "framework_type": "sentence_transformers"
    }
}
"""


# model_id_file_system = ml_client.register_model(model_path, model_config_path, model_group_id = "d4hfsYgBFp6IJxCcqpwi", isVerbose=True
model_id_file_system = ml_client.register_model(model_path, model_config_path, isVerbose=True)

Total number of chunks 10
Sha1 value of the model file:  3ead6e8725322ff54ef9137c453132046098d7e6494945283b8fc980c9123675
Model meta data was created successfully. Model Id:  4oh9sYgBFp6IJxCclpx2
uploading chunk 1 of 10
Model id: {'status': 'Uploaded'}
uploading chunk 2 of 10
Model id: {'status': 'Uploaded'}
uploading chunk 3 of 10
Model id: {'status': 'Uploaded'}
uploading chunk 4 of 10
Model id: {'status': 'Uploaded'}
uploading chunk 5 of 10
Model id: {'status': 'Uploaded'}
uploading chunk 6 of 10
Model id: {'status': 'Uploaded'}
uploading chunk 7 of 10
Model id: {'status': 'Uploaded'}
uploading chunk 8 of 10
Model id: {'status': 'Uploaded'}
uploading chunk 9 of 10
Model id: {'status': 'Uploaded'}
uploading chunk 10 of 10
Model id: {'status': 'Uploaded'}
Model registered successfully
Model deployed successfully


## Step 2: Load Model

In the last step we upload a model and the model id is: `t0jp24kBHUgZTTdfDMmm`. Now we will load this model in opensearch memory.

In [9]:
load_model_output = ml_client.deploy_model("t0jp24kBHUgZTTdfDMmm")

print(load_model_output)

Task ID: uEjr24kBHUgZTTdfNskV
Model deployed successfully
{'model_id': 't0jp24kBHUgZTTdfDMmm', 'task_type': 'DEPLOY_MODEL', 'function_name': 'TEXT_EMBEDDING', 'state': 'COMPLETED', 'worker_node': ['3w2aOHUBRmmwmVfoGgW8Jw'], 'create_time': 1691611772437, 'last_update_time': 1691611774664, 'is_async': True}


## Step 3: Get Task

When we invoke load model api of mlcommons plugin, a task get created. We can see the task id (`uEjr24kBHUgZTTdfNskV`) from previous output. Now, we can get the detailed information of the task using this task id

In [10]:
task_info = ml_client.get_task_info("uEjr24kBHUgZTTdfNskV")

print(task_info)

{'model_id': 't0jp24kBHUgZTTdfDMmm', 'task_type': 'DEPLOY_MODEL', 'function_name': 'TEXT_EMBEDDING', 'state': 'COMPLETED', 'worker_node': ['3w2aOHUBRmmwmVfoGgW8Jw'], 'create_time': 1691611772437, 'last_update_time': 1691611774664, 'is_async': True}


## Step 4: Get Model

With using the model id, we can also pull information about the model metadata from the opensearch cluster.

In [11]:
model_info = ml_client.get_model_info("t0jp24kBHUgZTTdfDMmm")

print(model_info)

{'name': 'huggingface/sentence-transformers/all-MiniLM-L12-v2', 'algorithm': 'TEXT_EMBEDDING', 'model_version': '1.0.1', 'model_format': 'TORCH_SCRIPT', 'model_state': 'DEPLOYED', 'model_content_size_in_bytes': 134568911, 'model_content_hash_value': 'f8012a4e6b5da1f556221a12160d080157039f077ab85a5f6b467a47247aad49', 'model_config': {'model_type': 'bert', 'embedding_dimension': 384, 'framework_type': 'SENTENCE_TRANSFORMERS', 'all_config': '{"_name_or_path":"microsoft/MiniLM-L12-H384-uncased","attention_probs_dropout_prob":0.1,"gradient_checkpointing":false,"hidden_act":"gelu","hidden_dropout_prob":0.1,"hidden_size":384,"initializer_range":0.02,"intermediate_size":1536,"layer_norm_eps":1e-12,"max_position_embeddings":512,"model_type":"bert","num_attention_heads":12,"num_hidden_layers":12,"pad_token_id":0,"position_embedding_type":"absolute","transformers_version":"4.8.2","type_vocab_size":2,"use_cache":true,"vocab_size":30522}'}, 'created_time': 1691611630758, 'last_updated_time': 169161

## Step 5: Generate Sentence Embedding

Now using the loaded model in memory, we can generate embedding for sentences. We can provide a list of sentences to get a list of embedding for the sentences. 

In [12]:
# Now using this model we can generate sentence embedding.

input_sentences = ["Test sentence1", "Test sentence2"]

embedding_output = ml_client.generate_embedding("t0jp24kBHUgZTTdfDMmm", input_sentences)

print(embedding_output)


{'inference_results': [{'output': [{'name': 'sentence_embedding', 'data_type': 'FLOAT32', 'shape': [384], 'data': [0.07004598, 0.094030194, 0.0294691, 0.0063353553, -0.037177887, 0.003469634, 0.06973778, -0.041374724, -0.052779537, -0.019993527, 0.049499035, 0.044301376, 0.050956327, -0.091860786, -0.039252494, -0.028518932, 0.018059155, -0.09713085, -0.034800865, 0.04408874, 0.025124485, -0.06829839, 0.02107054, 0.073358156, -0.016343018, 0.016885223, 0.0073822327, -0.069800906, 0.01917283, -0.127567, -0.002833644, 0.07662087, 0.010953967, 0.040559784, 0.047134362, -0.02965514, -0.025424507, -0.023706196, 0.015665805, -0.0002845036, -0.02252695, -0.073676944, 0.05547299, 0.018682828, 0.03940338, -0.024852037, 0.04160006, -0.0012200836, -0.012104877, -0.05119714, -0.07446669, -0.055452716, 0.007486106, -0.019089257, -0.030097323, -0.026060726, -0.052988816, 0.06712463, 0.025931384, -0.02644008, -0.0065706596, 0.05588654, -0.05347465, 0.007984704, 0.08091787, -0.036647186, -0.031903736,

## Step 6: Unload Model

After generating the embedding if we want we can unload the model from memory. `unload_model` method takes two input. 

1. model_id --> Which model we want to unload
2. node_ids --> list of the nodes from where we want to unload the model.

If we don't provide `node_ids` then method will unload model from all the nodes available like the following example.

In [13]:

undeploy_model_response = ml_client.undeploy_model("t0jp24kBHUgZTTdfDMmm")

print(undeploy_model_response)

{'3w2aOHUBRmmwmVfoGgW8Jw': {'stats': {'t0jp24kBHUgZTTdfDMmm': 'undeployed'}}}


## Step 7: Delete Model

We can also delete the model from the index using the model id.

In [14]:

delete_model_response = ml_client.delete_model("t0jp24kBHUgZTTdfDMmm")

print(delete_model_response)

{'_index': '.plugins-ml-model', '_id': 't0jp24kBHUgZTTdfDMmm', '_version': 6, 'result': 'deleted', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 11203, '_primary_term': 33}
